## Preparing the datasets

In [1]:
! wget https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/edgar_allan_poe.txt

--2022-11-30 20:21:58--  https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/edgar_allan_poe.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26622 (26K) [text/plain]
Saving to: ‘edgar_allan_poe.txt’

edgar_allan_poe.txt 100%[===================>]  26.00K  --.-KB/s    in 0.002s  

2022-11-30 20:21:58 (16.3 MB/s) - ‘edgar_allan_poe.txt’ saved [26622/26622]



In [6]:
! wget https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt

--2022-11-30 20:23:00--  https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 56286 (55K) [text/plain]
Saving to: ‘robert_frost.txt’

robert_frost.txt    100%[===================>]  54.97K  --.-KB/s    in 0.01s   

2022-11-30 20:23:00 (5.14 MB/s) - ‘robert_frost.txt’ saved [56286/56286]



In [92]:
# importing libraries
import pandas as pd
import numpy as np

In [93]:
#read the data 
with open("/content/edgar_allan_poe.txt", "r") as f:
  allan_poe = f.readlines()

with open("/content/robert_frost.txt", "r") as f:
  frost = f.readlines()

In [94]:
# building our dataframe and removing "\n"
df_poe = pd.DataFrame({"text":[text.lower().translate(str.maketrans(" "," ", string.punctuation)).replace("\n","") for text in allan_poe],"writer":"allan poe"})
df_frost = pd.DataFrame({"text":[text.lower().translate(str.maketrans(" "," ", string.punctuation)).replace("\n","") for text in frost],"writer":"frost"})


In [95]:
df = pd.concat([df_poe, df_frost])

In [96]:
# shuffel and split the data
shuffeled_df = df.sample(frac = 1)

In [97]:
len(df)*0.7

1664.6

In [98]:
#split
train = shuffeled_df[:1664]
test = shuffeled_df[1664:]

In [162]:
# Tokenize the text and mapping the words to index 
indx = 1
word2int = {0:"unk"}
for text in train["text"]:
  tokens = text.split()
  for token in tokens:
    if token not in word2int:
      word2int[indx] = token
      indx+=1

In [177]:
train_y = train["writer"]
test_y = test["writer"]

In [ ]:
list(word2int.values())

In [165]:
# vectorizer the samples
def to_word2int(df):
  li_docs = []
  for text in df["text"]:
    doc = []
    tokens = text.split()
    for token in tokens:
      if token in list(word2int.values()):
        doc.append(list(word2int.values()).index(token))
      else:
        doc.append(list(word2int.values()).index("unk"))

        
    li_docs.append(doc)
  return li_docs


In [166]:
train_as_int = to_word2int(train)
test_as_int = to_word2int(test)

In [168]:
["".join(word2int[indx]) for indx in vec_docs[10] ]

['and', 'talk', 'our', 'clearing', 'and', 'we']

In [72]:
import string

In [74]:
"sadjgölm2494ß2 34(/=$?f5a5g".translate(str.maketrans(" "," ", string.punctuation))

'sadjgölm2494ß2 34f5a5g'

## Building The Markov Classifier

In [169]:
V = len(word2int)
A0 = np.ones((V,V))
pi0 = np.ones((V))

A1 = np.ones((V,V))
pi1 = np.ones((V))


In [173]:
def compute_state_counts(text_as_int, A, pi):
  for tokens in text_as_int:
     last_indx = None
     for indx in tokens:
      if last_indx == None:
        pi[indx] +=1
      else:
        A[last_indx,indx]+=1
      last_indx=indx


In [181]:
compute_state_counts([x for (x,y) in zip(train_as_int, train_y) if y == "allan poe"] ,A0,pi0)
compute_state_counts([x for (x,y) in zip(train_as_int, train_y) if y == "frost"] ,A1,pi1)

5169